In [1]:
import collections
import numpy as np

import tensorflow as tf
import coremltools as ct

In [2]:
# get the first entry from 'squad_eval_mini.tfrecord'
filenames = ['tfrecords/squad_eval_mini.tfrecord']

raw_dataset = tf.data.TFRecordDataset(filenames, compression_type='ZLIB')
for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    
    result = {}
    # example.features.feature is the dictionary
    for key, feature in example.features.feature.items():
        # The values are the Feature objects which contain a `kind` which contains:
        # one of three fields: bytes_list, float_list, int64_list
        kind = feature.WhichOneof('kind')
        result[key] = np.array(getattr(feature, kind).value)
        
input_ids, input_mask, segment_ids = result['input_ids'], result['input_mask'], result['segment_ids']

In [3]:
# feed input_ids, input_mask, segment_ids into TF2 concrete function

tf.compat.v1.enable_resource_variables
tf_model = tf.saved_model.load('mobilebert_squad_savedmodels/float', tags='serve')
p = tf_model.prune(['input_ids:0', 'input_mask:0', 'segment_ids:0'], ['end_logits:0', 'start_logits:0'])
input_tensors = []
for i in [input_ids, input_mask, segment_ids]:
    input_tensors.append(tf.expand_dims(tf.convert_to_tensor(i, dtype=tf.int32), 0))
o1, o2 = p(input_tensors[0], input_tensors[1], input_tensors[2])

In [4]:
tf.math.top_k(o1, 10), tf.math.top_k(o2, 10)

(TopKV2(values=<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[ 5.618709 , -4.4534087, -5.0897   , -8.070566 , -8.493772 ,
         -8.609236 , -8.857419 , -9.096893 , -9.165716 , -9.285148 ]],
       dtype=float32)>, indices=<tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[47, 46, 58, 55, 61, 44, 41, 52, 43, 45]], dtype=int32)>),
 TopKV2(values=<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[ 6.004444 , -1.1120071, -4.0550504, -5.049522 , -5.7007403,
         -6.182735 , -6.478626 , -7.8853316, -8.23952  , -8.357329 ]],
       dtype=float32)>, indices=<tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[46, 47, 38, 57, 39, 48, 43, 50, 49, 58]], dtype=int32)>))

In [5]:
# feed input_ids, input_mask, segment_ids into Core ML model
# with all the compute units, we got different / unexpected results on devices with Neural Engine
# no surprise on macOS on x86_64 machines

model = ct.models.MLModel('MobileBERT.mlmodel')
inputs = {'input_ids': np.expand_dims(input_ids, 0).astype(np.float32), 'input_mask': np.expand_dims(input_mask, 0).astype(np.float32), 'segment_ids': np.expand_dims(segment_ids, 0).astype(np.float32)}

o = model.predict(inputs)

In [6]:
tf.math.top_k(o['start_logits'], 10), tf.math.top_k(o['end_logits'], 10)

(TopKV2(values=<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[ 6.0546875, -1.1083984, -4.0546875, -5.0585938, -5.671875 ,
         -6.1875   , -6.484375 , -7.9023438, -8.2578125, -8.359375 ]],
       dtype=float32)>, indices=<tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[46, 47, 38, 57, 39, 48, 43, 50, 49, 45]], dtype=int32)>),
 TopKV2(values=<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[ 3.0760000e+03,  3.6775000e+02,  3.3650000e+02,  1.6837500e+02,
          5.6640625e+00, -2.4133301e-01, -4.4648438e+00, -5.0898438e+00,
         -8.0859375e+00, -8.5234375e+00]], dtype=float32)>, indices=<tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[ 12,   4,   3, 375,  47, 373,  46,  58,  55,  61]], dtype=int32)>))

In [7]:
# feed input_ids, input_mask, segment_ids into Core ML model
# with CPU only

model = ct.models.MLModel('MobileBERT.mlmodel', compute_units=ct.ComputeUnit.CPU_ONLY)
o = model.predict(inputs)

In [8]:
tf.math.top_k(o['start_logits'], 10), tf.math.top_k(o['end_logits'], 10)

(TopKV2(values=<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[ 6.004442 , -1.1120118, -4.0550613, -5.0495315, -5.7007465,
         -6.1827283, -6.478644 , -7.88534  , -8.239529 , -8.357329 ]],
       dtype=float32)>, indices=<tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[46, 47, 38, 57, 39, 48, 43, 50, 49, 58]], dtype=int32)>),
 TopKV2(values=<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[ 5.618707 , -4.4534054, -5.089707 , -8.070569 , -8.4937725,
         -8.609242 , -8.857415 , -9.096891 , -9.16571  , -9.285147 ]],
       dtype=float32)>, indices=<tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[47, 46, 58, 55, 61, 44, 41, 52, 43, 45]], dtype=int32)>))